## Use SX5E and bloomberg download data to get me started

In [89]:
import pandas as pd
import numpy as np
from datetime import datetime


In [128]:
#initial cleaning function
def clean_sheet(dataframe):
    adj=dataframe.dropna(axis=1,how='all')
    adj=dataframe.set_index(adj.columns[0])
    return adj

In [152]:
#pull the data from excel
tabs_to_grab=['Quarter','Date','Reported EPS','Comp EPS','Consensus EPS','50d MA','3d return','Volume','Momentum',
             'Revision','Market cap','PE','property','Price']

data_dict={}

for s in tabs_to_grab:
    data_dict[s]=clean_sheet(pd.read_excel(r'C:\Users\Eric.Li\Documents\Post result code\Catalyst.xlsx',sheet_name=s))


In [302]:
#signal formation
signal=data_dict['Date'][(data_dict['3d return']>2) & (data_dict['Revision']>0.02)]
signal=signal.dropna(axis=1,how='all')
signal=signal.dropna(axis=0,how='all')

In [298]:
#Stack signals into a list of tuples
signal_list=[]
for index, row in signal.iterrows():
    adj_row=row.dropna()
    for s in adj_row.values:
        ts=(s - np.datetime64('1970-01-01T00:00:00Z')) / np.timedelta64(1, 's')
        adj_time=datetime.utcfromtimestamp(ts).strftime("%d/%b/%Y")
        signal_list.append((adj_row.name,adj_time))

C:\Users\Eric.Li\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  


In [299]:
# transform the datetime format
price=data_dict["Price"]
new_column=pd.to_datetime(price.columns).strftime("%d/%b/%Y").tolist()
price.columns=new_column

In [300]:
# create a dictionary to have signal lists as keys and return series as values
def signal_return_dict(signal_list,price_dataframe,holding):
    signal_dict={}
    for s in signal_list:
        price_series=price_dataframe.loc[s[0]]
        return_series=price_series.diff(1)/price_series
        day0=return_series.index.tolist().index(s[1])
        target_series=return_series.iloc[day0:day0+min(holding, len(return_series[day0:]))]
        target_series.index=range(min(holding, len(return_series[day0:])))
        signal_dict[s]=target_series

    return signal_dict

In [301]:
dict_test=signal_return_dict(signal_list,price,63)